In [1]:
%cd ..

/home/ubuntu/documents/dl_final/dl-final-project


# Parameters

In [2]:
batch_size = 16
hidden_size = 1024

unlabeled_epochs = 1
labeled_epochs = 1

variational = True

# Setup

In [3]:
import torch
import random
import numpy as np
from data import get_unlabeled_set, get_labeled_set, set_seeds, make_bounding_box_images, tensor_to_image
from model.resnet import Prototype

from time import perf_counter

In [4]:
set_seeds()
torch.backends.cudnn.benchmark = True

In [5]:
if torch.cuda.is_available():
    batch_size *= torch.cuda.device_count()

_, unlabeled_trainloader = get_unlabeled_set(batch_size=batch_size)
(_, labeled_trainloader), (_, labeled_testloader) = get_labeled_set(batch_size=batch_size, validation=0.2)

Validation Index: 128


In [6]:
# import matplotlib.pyplot as plt
# from helpers.helper import draw_box
# # The center of image is 400 * 400
# fig, ax = plt.subplots()
# color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']
# ax.imshow(road_image[0], cmap ='binary');
# # The ego car position
# ax.plot(400, 400, 'x', color="red")
# for i, bb in enumerate(target[0]['bounding_box']):
#     # You can check the implementation of the draw box to understand how it works
#     draw_box(ax, bb, color=color_list[target[0]['category'][i]])

# Model

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [8]:
model = Prototype(device, hidden_dim=hidden_size, variational=variational)

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model)
    assert batch_size >= torch.cuda.device_count()

model = model.to(device)

### Unlabeled Pre-training

In [9]:
criterion = model.module.var_loss_function
optimizer = torch.optim.Adam(model.parameters())

In [10]:
model.train()

start_time = perf_counter()
for epoch in range(unlabeled_epochs):
    loss = 0.0
    
    max_batches = len(unlabeled_trainloader)
    for idx, (images, camera_index) in enumerate(unlabeled_trainloader):
        optimizer.zero_grad()
        
        images = images.to(device)
        reconstructions, mu, logvar = model(images, mode='single-image')
        loss, mse, kld = criterion(reconstructions, images, mu, logvar, 'single-image',
                                  loss_reduction='sum')
        
        loss.backward()
        optimizer.step()
        
        # Training Wheels
        if idx != 0 and idx % 100 == 0:
            print('loss', loss.item())
            break

        if idx % 1000 == 0:
            print('[', epoch, '|', idx ,'/', max_batches, ']', 'loss:', loss.item(),
                 'curr time mins:', round(int(perf_counter() - start_time) / 60, 2))
            
#     model.save(file_prefix='unlabeled-')
    
print('Unlabeled Training Took (Min):', round(int(perf_counter() - start_time) / 60, 2))

[ 0 | 0 / 5009 ] loss: 484219.8125 curr time mins: 0.05
loss 239305.734375
Unlabeled Training Took (Min): 0.52


### Labeled Training

In [11]:
model.train()

start_time = perf_counter()
for epoch in range(labeled_epochs):
    loss = 0.0
    
    max_batches = len(labeled_trainloader)
    for idx, (images, targets, _) in enumerate(labeled_trainloader):
        optimizer.zero_grad()

        images = torch.stack(images).to(device)
        
        # Rasterize bounding box images for reconstruction
        targets = make_bounding_box_images(targets)
        targets = targets.to(device)
        
        # print('input shape:', images.shape)
        # print('targt shape:', targets.shape)
        
        reconstructions, mu, logvar = model(images, mode='object-map')
        
        # print('outpt shape:', reconstructions.shape)
        
        loss, ce, kld = criterion(reconstructions, targets, mu, logvar, 'object-map', 
                                  loss_reduction='sum')
        
        loss.backward()
        optimizer.step()
        
        # Training Wheels
        #print('loss', loss.item())
        #break
        
        if idx % 50 == 0:
            print('[', epoch, '|', idx ,'/', max_batches, ']', 'loss:', loss.item(),
                 'curr time mins:', round(int(perf_counter() - start_time) / 60, 2))
            
    Prototype.save(model, file_prefix='labeled-test-', using_dataparallel=True, epoch_num=1)
    
print('Labeled Training Took (Min):', round(int(perf_counter() - start_time) / 60, 2))

[ 0 | 0 / 174 ] loss: 1355311.625 curr time mins: 0.25
[ 0 | 50 / 174 ] loss: 757768.75 curr time mins: 2.42
[ 0 | 100 / 174 ] loss: 780738.5 curr time mins: 4.57
[ 0 | 150 / 174 ] loss: 854315.6875 curr time mins: 6.72
Labeled Training Took (Min): 7.88


# Inference

In [15]:
# device = 'cuda'
# d = torch.load('resnet_weights/labeled-test-resnet-1-epochs.torch', 
#            map_location=torch.device('cpu'))

# model = Prototype(device)
# model.load_state_dict(d)
# model.to(device)
# model.eval()

Prototype(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [16]:
(_, labeled_trainloader), (_, labeled_testloader) = get_labeled_set(batch_size=1, validation=0.2)
for (images, targets, _) in labeled_testloader:
    
    images = torch.stack(images).to(device)
    targets = make_bounding_box_images(targets).to(device)

    print(images.shape)
    print(targets.shape)
    
    img_recon, mu, logvar = model(images, mode='object-map')
    img_recon_sm = torch.argmax(torch.softmax(img_recon, 1), 1)
    print(img_recon_sm.shape)
    loss, ce, kld = criterion(img_recon, targets, mu, logvar, 'object-map', 
                                  loss_reduction='sum')
    
    print('loss', loss.item(), 'ce', ce.item(), 'kld', kld.item())
    print(np.unique(img_recon_sm.cpu().numpy()))

Validation Index: 128
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 48676.453125 ce 48672.7265625 kld 3.728271007537842
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 56036.83203125 ce 56033.84375 kld 2.9869470596313477
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 66200.0859375 ce 65725.890625 kld 474.1986083984375
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 56355.65625 ce 56353.046875 kld 2.6100611686706543
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 48736.5546875 ce 48733.91796875 kld 2.638319492340088
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 51912.83203125 ce 51909.078125 kld 3.7555336952209473
[0]
torch.Size([1, 6, 3, 256, 306])
torch.Size([1, 800, 800])
torch.Size([1, 800, 800])
loss 64188.56640625 

KeyboardInterrupt: 